<a href="https://colab.research.google.com/github/ferit-osirv/lab5/blob/main/lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.png" alt="Open In Colab"/></a>

# Lab 5 - Image Convolution

Ove laboratorijske vježbe se rješavaju u Google Colabu i spremaju na GitHub repozitorij koji je povezan na GitHub Classroom.

## Kako riješiti zadatke?

1. Prihvatite zadatak putem Google Classroom linka koji ćete dobiti. Google Classroom će kreirati repozitorij na vašem računu.
2. Uđite u novokreiran repozitorij na vašem računu i kliknite na **.ipynb** datoteku, zatim kliknite **Open in Colab**.
3. Zadatke rješavate u Google Colabu.

## Kako spremiti (predati) zadatke?

1. Unutar **Google Colaba** kliknite na **Open settings** kotačić u gornjem desnom kutu.
2. Kliknite na **GitHub** tab i odaberite kvačicu za **Access private repositories and organizations**.
3. Otvorit će se novi prozor da dodate pristup GitHubu. Kod **ferit-osirv** kliknite **Grant**.  
4. Spremite i izađite iz postavki.


5. Kliknite na **File > Save a copy in GitHub**.
6. Odaberite kreiran repozitorij labosa **koji uključuje vaše ime**.

> *Napomena:* Korake 1-4 morate napraviti samo prvi put.

## Kopiranje datoteka iz GitHub repozitorija

Za izradu vježbi bit će vam potrebne slike i druge datoteke koje će se nalaziti u GitHub repozitoriju vježbe. Ovakva komanda će biti dostupna u notebooku svake vježbe. Ona će kopirati datoteke s GitHuba u Google Colab okruženje.

**Ovu komandu je potrebno pokrenuti prije nego što krenete raditi svaku vježbu.**

In [ ]:
!rm -rf clone && git clone https://github.com/ferit-osirv/lab5 clone && cp -a clone/. .

## Introduction

![](images/conv.png)

Generally, convolution is a mathematical operation between two functions. In the context of this assignment, however, we will focus on discrete 2D convolution between two square images, as that is most relevant for image processing. Convolution is denoted as $I(A) \star k(B)$ where $I(A)$ is an image $I(A) \in \mathbb{R}^{W \times H}$ and $k(B)$ is a matrix $k(B) \in \mathbb{R}^{a \times b}$ indexed by locations $B \in \mathbb{N}^2$ called the **convolutional kernel**. At pixel $(x, y)$, the convolution operation is defined as:

\begin{equation}
(I \star k)(x, y) = \sum_{i=-a}^{a} \sum_{j=-b}^{b} I[x + i, y + j] k[i, j]
\end{equation}
 
Explained differently, the resulting image is produced by sliding the kernel over the input image pixel by pixel. At each pixel location, values where the kernel and the image overlap are multiplied, and all of the products are summed together to form the corresponding pixel's value in the output image.

![](images/no_padding_no_strides.gif)

While mathematically a simple operation, convolution is exceedingly powerful and can produce almost endless transformations of an image. It's most commonly used for filtering --- a convolution can elegantly find patterns in the image and increase their intensity. One such example is the convolution with a kernel called the Prewitt operator:

\begin{equation}
I_y(A) = I(A) \star \begin{bmatrix}
-1 & 0 & 1\\
-1 & 0 & 1\\
-1 & 0 & 1\\
\end{bmatrix}
\end{equation}

When convolved with this kernel, the resulting image has high-intensity pixels in regions where vertical edges are present, and low intensity everywhere else. This can be seen in the following image:

![](images/prewitt-example.png)

Vertical edges necessarily have to have a large jump in values going from left to right or right to left. Otherwise, there would be no perceptible edge. This kernel takes advantage of that fact to accentuate parts of the image where there is such a jump. It does this by replacing each pixel with the difference between the pixels on its left and its right.

This process happens as follows. For each pixel of the input image, the kernel is placed such that it is centered on that pixel. This means that the values of the pixel as well as its neighbors above and below are all multiplied by zero. The neighbors on the left are multiplied by -1, and the ones on the right are multiplied by 1. Summed together, the result represents the sum of the values on the right of the pixel, minus the sum of the values on the left.

To illustrate this, let us consider $1 \times 3$ region of the image where no vertical edges are present:

\begin{equation}
\begin{bmatrix}
128 & 130 & 136\\
\end{bmatrix}
\star
\begin{bmatrix}
-1 & 0 & 1\\
-1 & 0 & 1\\
-1 & 0 & 1\\
\end{bmatrix}
=
\sum_{i,j}
\begin{bmatrix}
-1 \times 0 + 0 \times 128 + 1 \times 130\\ 
-1 \times 128 + 0 \times 130 + 1 \times 136\\
-1 \times 130 + 0 \times 136 + 1 \times 0\\
\end{bmatrix}
= 8
\end{equation}

This section of the image does not contain a vertical edge, so the convolution result is a relatively low value. In a standard image with values in $[0, 255)$, 8 would appear almost completely black.

However, consider some section of the image where a vertical edge is indeed present:

\begin{equation}
\begin{bmatrix}
63 & 66 & 132\\
\end{bmatrix}
\star
\begin{bmatrix}
-1 & 0 & 1\\
-1 & 0 & 1\\
-1 & 0 & 1\\
\end{bmatrix}
=
\sum_{i,j}
\begin{bmatrix}
-1 \times 0 + 0 \times 64 + 1 \times 66\\ 
-1 \times 64 + 0 \times 66 + 1 \times 132\\
-1 \times 66 + 0 \times 132 + 1 \times 0\\
\end{bmatrix}
= 68
\end{equation}

The value is now much larger due to the difference between the left and right sides of the image. This example demonstrates how a relatively simple kernel can capture complex features of an image.

Beyond edge detection, there are many commonly used convolution kernels to perform tasks such as blurring, sharpening, or denoising images. A convolutional neural network can leverage the power of the convolution by stringing together sequences of intricate kernels to match complex patterns in the image.

### Resources

- https://en.wikipedia.org/wiki/Kernel_(image_processing)
- https://vincmazet.github.io/bip/filtering/convolution.html
- https://arxiv.org/pdf/1603.07285.pdf

## Exercises

We will first implement a convolution operation from scratch. Convert the following equation into code:

\begin{equation}
(I \star k)(x, y) = \sum_{i=1}^{a} \sum_{j=1}^{b} I[x + i, y + j] k[i, j].
\end{equation}

This equation defines the convolution value for a single pixel $(x, y)$, where $I$ is the image and $k$ is a kernel of size $a \times b$. You will need to perform this operation for each pixel. Complete `convolve_py` by returning a convolved image.

**Finish the code blocks below at the TODO comments. You do not need to change the rest of the code, or code blocks that do not have TODO comments.**

In [ ]:
def convolve_py(img, kernel):
  '''
  Convolves an image with a kernel.
  Uses pure Python (no Numpy).

  Args:
    img: 2D numpy array of the image (grayscale, e.g. (128,128))
    kernel: 2D numpy array of the kernel
  '''
  # Get the dimensions of the image and kernel
  img_height, img_width = img.shape
  kernel_height, kernel_width = kernel.shape

  conv_result = np.zeros(img.shape)
  img = np.pad(img, (1, 1), 'constant', constant_values=(0, 0))

  for x in range(img_width):
    for y in range(img_height):
      # TODO: compute pixel (x, y) using equation above
      # Note: If the kernel goes outside the boundary of the image, assume that the values outside the image are zero.
      # Note: In mathematical notation, matrix indices start at 1. In Python, matrix indices start at 0.
      # Note: The `img` pixel (x, y) is `img[y, x]`.

  return conv_result

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

img = cv.imread('images/cells.jpg', cv.IMREAD_GRAYSCALE)
plt.imshow(img, cmap='gray')
plt.show()
kernel = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])

img_out = convolve_py(img, kernel)
print(img_out.min(), img_out.max())
plt.imshow(img_out, cmap='gray')

### Speeding up convolution with numpy

This time, implement the equation above using Numpy functions on matrices to calculate the convolved pixel value as `np.sum(window * kernel)`.

In [ ]:
def convolve_np(img, kernel):
  '''
  Convolves an image with a kernel.
  Uses Numpy.

  Args:
    img: 2D numpy array of the image (grayscale)
    kernel: 2D numpy array of the kernel
  '''
  # Get the dimensions of the image and kernel
  img_height, img_width = img.shape
  kernel_height, kernel_width = kernel.shape

  conv_result = np.zeros(img.shape)
  img = np.pad(img, ((1, 1), (1, 1)), 'constant', constant_values=0)

  for x in range(img_width):
    for y in range(img_height):
      # TODO

  return conv_result

In [ ]:
img = cv.imread('images/cells.jpg', cv.IMREAD_GRAYSCALE)
img_out = convolve_np(img, kernel)
plt.imshow(img_out, cmap='gray')

Let us check which implementation is faster using the timeit library.

In [ ]:
import timeit

# Time the pure Python version
time_py = timeit.timeit('convolve_py(img, kernel)', number=5, globals=globals())
print('Pure Python: {:.5f}s'.format(time_py))

# Time the dot product version (should be a few seconds faster)
time_np = timeit.timeit('convolve_np(img, kernel)', number=5, globals=globals())
print('NumPy: {:.5f}s'.format(time_dot))

## Understanding kernels

Different convolutional kernels can achieve different effects such as:

 - edge detection
 - image engancement (e.g. sharpening)
 - blurring
 - denoising
 - feature detection

You can see various examples of kernels here:

 - https://en.wikipedia.org/wiki/Kernel_(image_processing)
 - https://setosa.io/ev/image-kernels/

In [ ]:
# TODO: Find a good convolutional kernel to perform edge detection on the cell image.

kernel_edge = # TODO
img_out = cv.filter2D(img, -1, kernel_edge)
plt.imshow(img_out, cmap='gray')

In [ ]:
# TODO: Find a kernel to blur the image and show the blurred image.
# Then, run the edge detection kernel on the blurred image and show the result.

kernel_blur = # TODO
# TODO: blur + edge detection, show the result

## Using large kernels

The next code block constructs an image. You job is to recreate the image using a convolution.

Hint: https://vincmazet.github.io/bip/filtering/convolution.html

In [ ]:
img_out = np.zeros((100, 100))
circle_origins = [(50, 50), (20, 20), (80, 20)]
circle_radius = 10

for (x, y) in circle_origins:
  img_out = cv.circle(img_out, (x, y), circle_radius, 1, -1)

plt.imshow(img_out, cmap='gray')
plt.show()

In [ ]:
kernel = np.zeros((circle_radius * 2 + 1, circle_radius * 2 + 1))
kernel = cv.circle(kernel, (circle_radius, circle_radius), circle_radius, 1, -1)
plt.imshow(kernel, cmap='gray')

img = np.zeros((100, 100))

# TODO: Modify `img` such that a convolution img * kernel produces the same result as `img_out`.


# OpenCV's built-in convolution function:
img = cv.filter2D(img, -1, kernel)
plt.imshow(img, cmap='gray')

## Using Convolutions for Feature Detection

Assume we want to detect the eyes on the following image:

![](images/woman_darkhair.png)

Notice that the eyes consist of a light region and then a dark region (iris). We may be able to detect the eyes using the following kernel:

\begin{bmatrix}
1 & 1 & -1 & -1\\
1 & 1 & -1 & -1\\
1 & 1 & -1 & -1\\
1 & 1 & -1 & -1\\
\end{bmatrix}

This kernel will take the difference between the left two pixels and the right two pixels. If the pixels on the left are high and on the right are low, the resulting value will be high.

Implement a convolution with this kernel in the next code block.

In [ ]:
img = cv.imread('images/woman_darkhair.png', cv.IMREAD_GRAYSCALE)
img = cv.resize(img, (0, 0), fx=0.5, fy=0.5)
plt.imshow(img, cmap='gray')
plt.show()

kernel = # TODO: Write the kernel from above as a numpy matrix

img_out = cv.filter2D(img, -1, kernel)
plt.imshow(img_out, cmap='gray')

Let's also find the mouth. Guided by the patterns you see in the pixel intensities, try to construct a kernel which will highlight the mouth in the image.

In [ ]:
img = cv.imread('images/woman_darkhair.tif', cv.IMREAD_GRAYSCALE)
cv.imwrite('images/woman_darkhair.png', img)

kernel = # TODO: Write a kernel to find the mouth

img_out = cv.filter2D(img, -1, kernel)
plt.imshow(img_out, cmap='gray')

Done! Don't forget to save the notebook on GitHub.